# BOMBAY STOCK EXCHANGE


### Recreation of Money Flow Index(MFI) and Moving AverageConvergence Divergence (MACD) metrics for listed companies using their stock price data

In [246]:
# import the libraries
import pandas as pd
import numpy as nm
import math
from numpy import log as ln
from datetime import datetime, date,timedelta
import datetime
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import yfinance as yf # retrieves data from yahoo finance site 

Getting the data using yfinance library

In [247]:
df = yf.Ticker('HINDUNILVR.NS').history(period='3y')[['Close', 'Open', 'High', 'Volume', 'Low']]

In [248]:
df=df.reset_index()
df

,Date,Close,Open,High,Volume,Low
0,2019-07-15,1627.145386,1625.297426,1634.490024,725180,1612.977387
1,2019-07-16,1650.221924,1633.352901,1659.035421,1259786,1623.165176
2,2019-07-17,1668.607178,1648.989930,1673.203593,1278232,1635.580018
3,2019-07-18,1648.184448,1673.630046,1673.630046,1006299,1619.611409
4,2019-07-19,1630.415039,1656.571407,1669.791710,983217,1619.753467
...,...,...,...,...,...,...
741,2022-07-11,2490.600098,2487.000000,2525.000000,1587197,2480.000000
742,2022-07-12,2449.550049,2475.000000,2490.600098,1116476,2444.449951
743,2022-07-13,2497.750000,2464.000000,2519.350098,2528792,2461.399902
744,2022-07-14,2497.800049,2502.750000,2522.899902,1663307,2468.199951


### Calculation of MFI 
We first have to calculate the Typical Price and Raw money flow from the drataframe before calculating MFI.

#### Typical Price=( Close +Open+High)/3

#### Raw Money Flow= Typical Price * Volume

#### Money Flow Ratio= 14 Period Positive Money Flow/14 Period Negative Money Flow

#### MFI= 100− (100/1+Money Flow Ratio)
 


In [249]:
df1=pd.DataFrame()
df1['Date']=df['Date']
df1['Typical Price']=((df['Close']+df['Open']+df['High'])/3)
df1['Raw Money Flow']=df1['Typical Price']*df['Volume']
df1

,Date,Typical Price,Raw Money Flow
0,2019-07-15,1628.977612,1.181302e+09
1,2019-07-16,1647.536748,2.075544e+09
2,2019-07-17,1663.600234,2.126467e+09
3,2019-07-18,1665.148180,1.675637e+09
4,2019-07-19,1652.259385,1.624530e+09
...,...,...,...
741,2022-07-11,2500.866699,3.969368e+09
742,2022-07-12,2471.716715,2.759612e+09
743,2022-07-13,2493.700033,6.306049e+09
744,2022-07-14,2507.816650,4.171269e+09


In [250]:
end_date= df1['Date'].max()
start_date= df1['Date'].min()

In [251]:
df2=df
df=df1

In [252]:
# Get all of the positive and negative money flows

positive_flow = []
negative_flow = []

# Loop through the typical price
for i in range(1, len(df)):
    if df['Typical Price'][i] > df['Typical Price'][i-1]:
        positive_flow.append( df['Raw Money Flow'][i-1])
        negative_flow.append(0)
        
    elif df['Typical Price'][i] < df['Typical Price'][i-1]:
        negative_flow.append(df['Raw Money Flow'][i-1])
        positive_flow.append(0)
        
    else:
        positive_flow.append(0)
        negative_flow.append(0)

In [253]:
period=14

positive_mf = []
negative_mf = []

for i in range(period-1, len(positive_flow)):
    positive_mf.append( sum(positive_flow[i + 1- period : i+1]))
    
for i in range(period-1, len(negative_flow)):
    negative_mf.append( sum(negative_flow[i + 1- period : i+1]))
    

# Calculate the money flow index
MFI = 100 * (nm.array(positive_mf) / (nm.array(positive_mf) + nm.array(negative_mf) ))


In [254]:
df1=pd.DataFrame()
df1['Date']=df['Date']
df1['Stock Price']=df2['Close']
df1['MFI']= 'NaN'
df1['MFI'][14:]=MFI

In [255]:
# Code to visualize MFI in Python.

# df3 = pd.DataFrame()
# df3['MFI'] = MFI

# # Create the plot

# plt.figure(figsize=(15,12))
# plt.plot(df2['MFI'], label='MFI')
# plt.axhline(10, linestyle= '--', color='orange')
# plt.axhline(20, linestyle= '--', color='blue')
# plt.axhline(80, linestyle= '--', color='blue')
# plt.axhline(90, linestyle= '--', color='orange')
# plt.title('MFI', fontsize=18)
# plt.ylabel('MFI Values')
# plt.show()

### Calculation of MACD

In [256]:
df = yf.Ticker('HINDUNILVR.NS').history(period='3y')[['Close', 'Open', 'High', 'Volume', 'Low']]

In [257]:
# Get the 12-day EMA of the closing price
k = df['Close'].ewm(span=12, adjust=False, min_periods=12).mean()

# Get the 26-day EMA of the closing price
d = df['Close'].ewm(span=26, adjust=False, min_periods=26).mean()
# Subtract the 26-day EMA from the 12-Day EMA to get the MACD
macd = k - d

# Get the 9-Day EMA of the MACD for the Trigger line
macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()

# Add all of our new values for the MACD to the dataframe
df['macd'] = df.index.map(macd)
df['macd_s'] = df.index.map(macd_s)
df=df.reset_index()
# View our data

df1['MACD']=df['macd'] # adding to df1(final dataframe)

df1['EXP(9)']=df['macd_s']

In [258]:
df1

,Date,Stock Price,MFI,MACD,EXP(9)
0,2019-07-15,1627.145386,NaN,NaN,NaN
1,2019-07-16,1650.221924,NaN,NaN,NaN
2,2019-07-17,1668.607178,NaN,NaN,NaN
3,2019-07-18,1648.184448,NaN,NaN,NaN
4,2019-07-19,1630.415039,NaN,NaN,NaN
...,...,...,...,...,...
741,2022-07-11,2490.600098,67.607469,73.752835,47.622856
742,2022-07-12,2449.550049,67.199798,73.427446,52.783774
743,2022-07-13,2497.750000,66.680862,76.180742,57.463167
744,2022-07-14,2497.800049,67.971982,77.473719,61.465278


In [259]:
df1.to_csv("C:\\Users\\archi\\OneDrive\\Desktop\\bse_intern\\Assignment5\\tech_an_output.csv")